In [1]:
"""ATE: Automatic Term Extraction
"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import MeCabTokenizer


In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [3]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [6]:
print("\n".join(docs[2023][:15]))

生体医工学ウェブ辞典（第二分冊）
医療機関等への建築ガイドライン浸透のための課題と今後の展望
メーカーから見た医用テレメータ設置・運用に関する現状と確認事項
医用テレメータの概要と医療現場での現状と希望
医療機関における電波利用機器に配慮した建築ガイドライン・同解説－医用テレメータ編－のポイント
生体医工学×医療者教育―効率的な知識・技術発展のためにー
生体医工学×知のデザイン －臨床研究における学際の必要性－
生体医工学×医工連携～自身の開発経験談～
生体医工学×医療機器管理のエビデンス創設を目指して
スマートコントラクトを用いたトラストデータ流通システム
聴診音解析システムを活用した医療DXへの挑戦
看護理工学的思考によるリアルワールドデータの活用
皮膚感覚メカニズム解明のためのセンシング・ディスプレイ技術
神経オルガノイドを用いた特殊培養デバイスによる疼痛評価システム
次世代網膜刺激型人工視覚デバイスの開発


In [7]:
year = 2019

tok = MeCabTokenizer() 
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題',"および", "評価"]
ignores1 = ['型']
ignores2 = ['中','的','時','システム','用']

compound_nouns = []
for y in years:
    for sentence in docs[y]:
        if sentence.isascii():
            continue 
    
        tokens = tok.iter_token(sentence)
        nouns = tok.extract_nouns(tokens)
        # print(nouns)
        nouns = tok.filter_nouns(nouns, n=1, stopwords=ignores, stopwords_begin=ignores1, stopwords_end=ignores2)
        # print(nouns)
        if len(nouns) > 0 :
            compound_nouns.extend(nouns)

lr = LRValue()
# lr_values = lr.fit_transform(compound_nouns)
lr_values = lr.fit(compound_nouns).transform(compound_nouns, mode=3)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)
list(data_collection)[:50]

/tmp/working/notebooks/theme_river/termextract/termextract.py:53: RuntimeWarning: invalid value encountered in power
  return np.power(a.cumprod()[-1], 1/(2 * len(compound_noun)))


[('-', 134.81839637082174),
 ('性', 104.69479452198185),
 ('計測 -', 104.35654867978765),
 ('計測', 80.77747210701756),
 ('画像 計測', 71.4938436471975),
 ('機能 計測', 69.63720385900268),
 ('細胞', 68.70225614927067),
 ('推定 -', 65.42439443697378),
 ('細胞 機能', 64.22168487654599),
 ('細胞 運動', 63.88040248230294),
 ('画像', 63.2771680782255),
 ('計測 解析', 62.7753825941037),
 ('細胞 モデル', 61.70790503529301),
 ('計測 光', 60.95706453662945),
 ('光 計測', 60.95706453662945),
 ('脳 機能 計測', 60.160204567016834),
 ('機能', 60.03332407921454),
 ('生体 画像 計測', 59.97869985493808),
 ('超音波 画像 計測', 59.854647220041066),
 ('運動 機能', 59.71429917951737),
 ('運動', 59.39696961966999),
 ('生体 機能 計測', 58.935752495514805),
 ('生体 計測', 58.394514758537674),
 ('細胞 解析', 57.89349105868703),
 ('位置 -', 57.20097114454519),
 ('細胞 組織', 57.11155401954018),
 ('画像 刺激', 57.01964366370913),
 ('刺激 画像', 57.01964366370913),
 ('光 脳 機能 計測', 56.25633512211709),
 ('モデル', 55.42562584220407),
 ('血管 細胞', 54.39656526759669),
 ('呼吸 機能 計測', 54.38256203338508),
 ('自動 計測', 53.

In [8]:
terms = lr_values
freq = lr.compound_noun
terms.sort(key=lambda z: (len(z[0].split()), len(z[0]),-z[1]), reverse=False)
n, m = 2100, 20
for i, v in terms[n:n+m]:
    print(i, v, freq[i])

インタフェース 3.0 1
スマートフォン 2.449489742783178 16
マイクロフォン 2.449489742783178 1
パルスジェット 2.449489742783178 1
パーキンソン病 2.23606797749979 1
ホルター心電図 2.0 4
ミトコンドリア 2.0 6
インフルエンザ 2.0 1
トモグラフィー 2.0 2
MIMOSYS 1.7320508075688772 2
フォースセンサ 1.7320508075688772 1
IEEE802 1.4142135623730951 1
アストロサイト 1.4142135623730951 1
変形性膝関節症 1.4142135623730951 2
インターネット 1.4142135623730951 1
サーモグラフィ 1.4142135623730951 2
マーケティング 1.4142135623730951 2
マクロファージ 1.4142135623730951 1
マイクロバブル 1.0 1
モンテカルロ法 1.0 2
